In [18]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory,Xception_reduced,dot_distance_neg,\
                        TripletGenerator,create_model_triplet
from utility.grayscale_imagenet import Xception_greyscale
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False
shapes = (224,224,1)
HalfBatch = 8
margin = -2

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = TripletGenerator(Ids_train,newWhale_train,transform,HalfBatch=HalfBatch)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test,HalfBatch=HalfBatch)

### build model

In [5]:
conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
#conv_base = Xception_greyscale((256,256,1),'max',False)
# conv_base = Xception_reduced((256,256,1))

In [6]:
train_model,feature_model = create_model_triplet(1e-3,l2_distance,\
                                                 conv_base,color,margin,BatchNorm=True,\
                                                nodes=[512,256], activations=['relu',None])

### training

In [7]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
713/713 [==============================] - 52s 73ms/step - loss: -1.1605 - val_loss: -1.2011
Epoch 2/5
713/713 [==============================] - 42s 60ms/step - loss: -1.2641 - val_loss: -1.1576
Epoch 3/5
713/713 [==============================] - 43s 60ms/step - loss: -1.3239 - val_loss: -1.3509
Epoch 4/5
713/713 [==============================] - 43s 60ms/step - loss: -1.3729 - val_loss: -1.3159
Epoch 5/5
713/713 [==============================] - 43s 60ms/step - loss: -1.3547 - val_loss: -1.2789
time:224.0328893661499


Fine-Tune

In [8]:
feature_model.layers[0].trainable=True

In [9]:
train = False
for layer in feature_model.layers[0].layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

In [10]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=4e-4))

In [11]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
713/713 [==============================] - 72s 101ms/step - loss: -1.4453 - val_loss: -1.2905
Epoch 2/5
713/713 [==============================] - 56s 79ms/step - loss: -1.5949 - val_loss: -1.3916
Epoch 3/5
713/713 [==============================] - 56s 79ms/step - loss: -1.6373 - val_loss: -1.3688
Epoch 4/5
713/713 [==============================] - 56s 79ms/step - loss: -1.6363 - val_loss: -1.4364
Epoch 5/5
713/713 [==============================] - 56s 79ms/step - loss: -1.6536 - val_loss: -1.4600
time:302.4082691669464


In [12]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=2,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/2
713/713 [==============================] - 56s 79ms/step - loss: -1.7069 - val_loss: -1.4639
Epoch 2/2
713/713 [==============================] - 56s 79ms/step - loss: -1.6914 - val_loss: -1.4143
time:112.09834861755371


In [13]:
train = False
for layer in feature_model.layers[0].layers:
    if 'conv4' in layer.name:
        train = True
    layer.trainable = train

In [14]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=1e-4))

In [15]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
713/713 [==============================] - 109s 153ms/step - loss: -1.6969 - val_loss: -1.5322
Epoch 2/5
713/713 [==============================] - 81s 113ms/step - loss: -1.7570 - val_loss: -1.5923
Epoch 3/5
713/713 [==============================] - 81s 113ms/step - loss: -1.7953 - val_loss: -1.6548
Epoch 4/5
713/713 [==============================] - 81s 113ms/step - loss: -1.8070 - val_loss: -1.6588
Epoch 5/5
713/713 [==============================] - 81s 114ms/step - loss: -1.8143 - val_loss: -1.6298
time:445.4922471046448


In [16]:
train = True
for layer in feature_model.layers[0].layers:
    layer.trainable = train

In [17]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=1e-5))

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=1e-10)

In [20]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=16,
          use_multiprocessing=True,workers=2,max_queue_size=20,
          callbacks=[reduce_lr])
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/16
713/713 [==============================] - 172s 241ms/step - loss: -1.8562 - val_loss: -1.7805
Epoch 2/16
713/713 [==============================] - 130s 182ms/step - loss: -1.8795 - val_loss: -1.7709
Epoch 3/16
713/713 [==============================] - 130s 183ms/step - loss: -1.8680 - val_loss: -1.7497
Epoch 4/16
713/713 [==============================] - 130s 183ms/step - loss: -1.8895 - val_loss: -1.7391
Epoch 5/16
713/713 [==============================] - 130s 183ms/step - loss: -1.8950 - val_loss: -1.8291
Epoch 6/16
713/713 [==============================] - 130s 182ms/step - loss: -1.8981 - val_loss: -1.8258
Epoch 7/16
713/713 [==============================] - 130s 182ms/step - loss: -1.9032 - val_loss: -1.7895
Epoch 8/16
713/713 [==============================] - 130s 182ms/step - loss: -1.8944 - val_loss: -1.8008
Epoch 9/16
713/713 [==============================] - 137s 192ms/step - loss: -1.9185 - val_loss: -1.8042
Epoch 10/16
713/713 [=========================

In [21]:
train_model.save('Models/DenseNet_Triplet_BN_MLP.h5')
feature_model.save('Models/feature_model_DenseNet_Triplet_BN_MLP.h5')